# <center>Critical AI</center>
<center>ENGL 54.41</center>
<center>Dartmouth College</center>
<center>Winter 2026</center>
<pre>Created: 02/18/2026

## NOTE

We want to run this in a GPU environment (e.g., T4 or better).

In [ ]:
# install FAISS first
!pip install faiss-cpu > /dev/null 2>&1
from openai import OpenAI
import torch
import faiss
import json
from sentence_transformers import SentenceTransformer
import os
import numpy as np
import pandas as pd

In [ ]:
model_name = "anthropic.claude-opus-4-6"
api_key = "API_KEY_GOES_HERE"

client = OpenAI(base_url="https://chat.dartmouth.edu/api", 
                api_key=api_key)

In [ ]:
# This cell of code will determine if we have an accelerator for running
# our neural networks.
# mps == Apple Silicon device (MX series of Macbooks)
# cuda == Compute Unified Device Architecture is a toolkit from Nvidia and means we have a GPU
# cpu == Just using the general-purpose CPU for our calculations

if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('Using device: {0}'.format(device))

In [ ]:
# download LayupList data
!wget -O old_reviews.json https://raw.githubusercontent.com/jeddobson/ENGL64.05-22F/refs/heads/main/data/LayupList/old_reviews.json

In [ ]:
# open older format reviews and extract comments
reviews = json.loads(open("old_reviews.json").read())
reviews_text= [r["comments"]["oldReview"] for r in reviews if 'comments' in r]
# how many did we find?
print("found {0} reviews".format(len(reviews_text)))

In [ ]:
# load a smaller embedding model that will quickly embed all our documents
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# move model to our accelerator device
embedding_model.to(device)

In [ ]:
# Create document embeddings with embedding model
doc_embeddings = embedding_model.encode(reviews_text)

In [ ]:
# display number of documents and embedding width
doc_embeddings.shape

In [ ]:
# Build FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# this will retrieve five closest neighbors using document similarity
def retrieve_documents(query, k=5):
    query_embedding = embedding_model.encode([query])[0]
    distances, indices = index.search(np.array([query_embedding]), k)
    return [reviews_text[i] for i in indices[0]]

In [ ]:
query = "I am interested in the very best courses offered in the Spanish Department. Who are the best professors in Spanish?"
query_embedding = embedding_model.encode([query])[0]
distances, indices = index.search(np.array([query_embedding]), 25)

In [ ]:
df = pd.DataFrame({"Similarity":distances[0],
                   "Document Idx":indices[0],
                   "Contents":[reviews_text[i][:40].replace('\t','') for i in indices[0]]})
df.sort_values(by='Similarity',ascending=False)

In [ ]:
query = "I am interested in the very best courses offered in the Spanish Department. Who are the best professors in Spanish?"

# Retrieve relevant documents for our query
retrieved_docs = retrieve_documents(query,k=25)

# Join query with context.
context = query + "\nCourse Review Data:\n" + "\n".join(retrieved_docs)

In [ ]:
context

In [ ]:
output = client.chat.completions.create(
    model = model_name,
    messages = [
        {"role": "system", "content": "You are a helpful assistant. You are also a chatbot able to recommend courses to Dartmouth students based on reviews." },
        {"role": "user", "content": context}
    ],
    stream = False)

In [ ]:
# display "usage" information. How many tokens were generated? How many did we submit 
# as part of prompt?
output.usage

In [ ]:
# message content is the output after the end of the reasoning trace.
response = output.choices[0].message.content

In [ ]:
print(response)